## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from vacation_search_config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,32.67,67,100,3.09,overcast clouds
1,1,Bengkulu,ID,-3.8004,102.2655,77.25,83,48,4.16,scattered clouds
2,2,Cap Malheureux,MU,-19.9842,57.6142,79.29,86,100,1.99,overcast clouds
3,3,Mount Gambier,AU,-37.8333,140.7667,84.11,42,40,9.22,light rain
4,4,Punta Arenas,CL,-53.1500,-70.9167,50.11,46,40,23.02,scattered clouds


In [28]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60


In [29]:
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the maximum temperature you would like for your trip? 100


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bengkulu,ID,-3.8004,102.2655,77.25,83,48,4.16,scattered clouds
2,2,Cap Malheureux,MU,-19.9842,57.6142,79.29,86,100,1.99,overcast clouds
3,3,Mount Gambier,AU,-37.8333,140.7667,84.11,42,40,9.22,light rain
5,5,Colon,PA,9.3592,-79.9014,79.25,78,7,9.82,clear sky
6,6,Viedma,AR,-40.8135,-62.9967,92.07,18,0,10.87,clear sky
9,9,Quatre Cocos,MU,-20.2078,57.7625,78.91,86,100,8.41,overcast clouds
10,10,Ninh Binh,VN,20.2539,105.9750,63.70,86,100,8.39,overcast clouds
11,11,Vaini,TO,-21.2000,-175.2000,86.16,89,75,12.66,broken clouds
12,12,Antalaha,MG,-14.9003,50.2788,75.65,89,100,4.94,overcast clouds
14,14,Busselton,AU,-33.6500,115.3333,69.04,62,17,12.55,few clouds


In [31]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                7
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
nonEmpty_cities_df = preferred_cities_df.dropna()
nonEmpty_cities_df.count()

City_ID                345
City                   345
Country                345
Lat                    345
Lng                    345
Max Temp               345
Humidity               345
Cloudiness             345
Wind Speed             345
Current Description    345
dtype: int64

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = nonEmpty_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,77.25,scattered clouds,-3.8004,102.2655,
2,Cap Malheureux,MU,79.29,overcast clouds,-19.9842,57.6142,
3,Mount Gambier,AU,84.11,light rain,-37.8333,140.7667,
5,Colon,PA,79.25,clear sky,9.3592,-79.9014,
6,Viedma,AR,92.07,clear sky,-40.8135,-62.9967,
9,Quatre Cocos,MU,78.91,overcast clouds,-20.2078,57.7625,
10,Ninh Binh,VN,63.70,overcast clouds,20.2539,105.9750,
11,Vaini,TO,86.16,broken clouds,-21.2000,-175.2000,
12,Antalaha,MG,75.65,overcast clouds,-14.9003,50.2788,
14,Busselton,AU,69.04,few clouds,-33.6500,115.3333,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, hotel in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    latitude = hotel["Lat"]
    longitude = hotel["Lng"]
    params["location"] =  f"{latitude},{longitude}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

print("End of loop.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
End of loop.


In [42]:
# 7. Drop the rows where there is no Hotel Name.
# first find rows that are empty
hotel_df[hotel_df['Hotel Name'] == ''] 
# next find rows that arent empty
hotel_df[hotel_df['Hotel Name'] != ''] 
# set the hotel dataframe to non empty hotel names
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df.count()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,77.25,scattered clouds,-3.8004,102.2655,Grage Hotel Bengkulu
2,Cap Malheureux,MU,79.29,overcast clouds,-19.9842,57.6142,Pereybere Hotel & Spa
3,Mount Gambier,AU,84.11,light rain,-37.8333,140.7667,The Commodore
5,Colon,PA,79.25,clear sky,9.3592,-79.9014,Sand Diamond Hotel
6,Viedma,AR,92.07,clear sky,-40.8135,-62.9967,Hotel Cristal
9,Quatre Cocos,MU,78.91,overcast clouds,-20.2078,57.7625,Sunrise Attitude Hotel
10,Ninh Binh,VN,63.70,overcast clouds,20.2539,105.9750,Kinh Do Hotel
11,Vaini,TO,86.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Antalaha,MG,75.65,overcast clouds,-14.9003,50.2788,Hotel Vitasoa
14,Busselton,AU,69.04,few clouds,-33.6500,115.3333,Observatory Guest House


In [44]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,77.25,scattered clouds,-3.8004,102.2655,Grage Hotel Bengkulu
2,Cap Malheureux,MU,79.29,overcast clouds,-19.9842,57.6142,Pereybere Hotel & Spa
3,Mount Gambier,AU,84.11,light rain,-37.8333,140.7667,The Commodore
5,Colon,PA,79.25,clear sky,9.3592,-79.9014,Sand Diamond Hotel
6,Viedma,AR,92.07,clear sky,-40.8135,-62.9967,Hotel Cristal
9,Quatre Cocos,MU,78.91,overcast clouds,-20.2078,57.7625,Sunrise Attitude Hotel
10,Ninh Binh,VN,63.70,overcast clouds,20.2539,105.9750,Kinh Do Hotel
11,Vaini,TO,86.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Antalaha,MG,75.65,overcast clouds,-14.9003,50.2788,Hotel Vitasoa
14,Busselton,AU,69.04,few clouds,-33.6500,115.3333,Observatory Guest House


In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))